# 🚀 Ailo Network - Colab GPU Miner

**Mine AiloCoin usando la GPU gratuita di Google Colab!**

Questo notebook usa **AILO-1B** - lo stesso modello del CUDA Miner ufficiale.

## ⚠️ IMPORTANTE: Prima di iniziare
1. Vai su **Runtime > Disconnect and delete runtime**
2. Poi **Runtime > Run all** per partire pulito

## Requisiti:
- Account Ailo Network (https://ailo.site)
- GPU T4 abilitata (Runtime > Cambia tipo di runtime > GPU)

---

In [ ]:
#@title ⚙️ 1. Setup - Installa Dipendenze e Pulisci Memoria

# CRITICAL: Clean up any existing GPU memory from previous runs
import gc
gc.collect()

try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
        for obj in gc.get_objects():
            if torch.is_tensor(obj):
                del obj
        gc.collect()
        torch.cuda.empty_cache()
except:
    pass

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
!pip install websockets aiohttp requests numpy tqdm -q

import torch
import gc

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

print(f"✅ PyTorch {torch.__version__}")
print(f"✅ CUDA disponibile: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
    allocated = torch.cuda.memory_allocated() / 1024**3
    print(f"✅ VRAM: {vram:.1f} GB (used: {allocated:.2f} GB)")

In [ ]:
#@title 🔑 2. Inserisci il tuo Wallet Address
WALLET_ADDRESS = ""  #@param {type:"string"}

if not WALLET_ADDRESS or len(WALLET_ADDRESS) < 40:
    print("❌ Inserisci un wallet address valido!")
    print("   Puoi trovarlo su https://ailo.site/wallet.html")
else:
    print(f"✅ Wallet: {WALLET_ADDRESS[:12]}...{WALLET_ADDRESS[-8:]}")

In [ ]:
#@title 🧠 3. AILO-1B Training Engine

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import asyncio
import websockets
import aiohttp
import requests
import json
import time
import base64
import gzip
import gc
from datetime import datetime
from typing import Optional, Dict, Tuple, List

# Configuration
SERVER_URL = "https://ailo.site"
WS_URL = "wss://ailo.site/ws/cuda"
API_URL = "https://ailo.site/api"
CLIENT_VERSION = "1.2.0-colab"
HEARTBEAT_INTERVAL = 10
SUBMIT_INTERVAL = 300

# T4 Settings - ultra memory efficient
DEFAULT_BATCH_SIZE = 1
DEFAULT_SEQ_LEN = 32
GRADIENT_ACCUMULATION = 16
DEFAULT_LEARNING_RATE = 0.001

class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size=50257, d_model=1600, nhead=25, num_layers=24, dim_feedforward=6400):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = nn.Parameter(torch.zeros(1, 512, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=0.1, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, vocab_size)
        self.d_model = d_model
    
    def forward(self, x):
        seq_len = x.size(1)
        x = self.embedding(x) * np.sqrt(self.d_model)
        x = x + self.pos_encoding[:, :seq_len, :]
        x = self.transformer(x)
        return self.fc_out(x)

class TrainingEngine:
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = None
        self.optimizer = None
        self.criterion = nn.CrossEntropyLoss()
        self.total_steps = 0
        self.best_loss = float('inf')
        self.tokens_processed = 0
        self.accumulated_steps = 0
    
    def initialize_model(self):
        print("🧠 Initializing AILO-1B model...")
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        print("   📐 Using AILO-1B: 24 layers × 1600d")
        self.model = SimpleTransformer().half().to(self.device)
        
        total_params = sum(p.numel() for p in self.model.parameters())
        print(f"   Parameters: {total_params:,} ({total_params/1e9:.2f}B)")
        
        if torch.cuda.is_available():
            print(f"   💾 Model memory: {torch.cuda.memory_allocated() / 1024**3:.1f} GB")
        
        self.optimizer = optim.SGD(self.model.parameters(), lr=DEFAULT_LEARNING_RATE, momentum=0.9, weight_decay=0.01)
        print("   ✅ Using SGD optimizer (memory efficient)")
        
        for name, param in self.model.named_parameters():
            if 'weight' in name and param.dim() > 1:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)
        
        print(f"   ⚡ batch={DEFAULT_BATCH_SIZE}, seq={DEFAULT_SEQ_LEN}, accum={GRADIENT_ACCUMULATION}")
        return self.model
    
    def _fetch_real_data(self, count, wallet):
        try:
            response = requests.get(f"{SERVER_URL}/api/cuda/training-data", params={'batchSize': count, 'wallet': wallet}, headers={'User-Agent': f'AiloMiner/{CLIENT_VERSION}'}, timeout=30)
            if response.status_code == 200:
                data = response.json()
                if data.get('articles'):
                    return data['articles']
        except:
            pass
        return ["Machine learning is AI. Deep learning uses neural networks. Transformers use attention."] * count
    
    def generate_training_batch(self, wallet):
        vocab_size = 50257
        texts = self._fetch_real_data(DEFAULT_BATCH_SIZE, wallet)
        batch_x, batch_y = [], []
        for text in texts:
            tokens = [ord(c) % vocab_size for c in text[:DEFAULT_SEQ_LEN + 1]]
            while len(tokens) < DEFAULT_SEQ_LEN + 1:
                tokens.append(0)
            batch_x.append(tokens[:DEFAULT_SEQ_LEN])
            batch_y.append(tokens[1:DEFAULT_SEQ_LEN + 1])
        return torch.tensor(batch_x, dtype=torch.long), torch.tensor(batch_y, dtype=torch.long)
    
    def train_step(self, batch_x, batch_y):
        self.model.train()
        batch_x = batch_x.to(self.device)
        batch_y = batch_y.to(self.device)
        
        outputs = self.model(batch_x)
        loss = self.criterion(outputs.view(-1, outputs.size(-1)), batch_y.view(-1)) / GRADIENT_ACCUMULATION
        loss.backward()
        self.accumulated_steps += 1
        
        if self.accumulated_steps >= GRADIENT_ACCUMULATION:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            self.optimizer.zero_grad(set_to_none=True)
            self.accumulated_steps = 0
        
        tokens = batch_x.numel()
        self.tokens_processed += tokens
        self.total_steps += 1
        
        if self.total_steps % 200 == 0:
            torch.cuda.empty_cache()
        
        return loss.item() * GRADIENT_ACCUMULATION, tokens
    
    def extract_gradients(self):
        return {name: param.grad.clone() for name, param in self.model.named_parameters() if param.grad is not None}
    
    def compress_gradients(self, gradients):
        flat_grads = [gradients[name].cpu().float().flatten() for name in sorted(gradients.keys())]
        all_grads = torch.cat(flat_grads).half()
        grad_bytes = all_grads.numpy().tobytes()
        compressed = gzip.compress(grad_bytes, compresslevel=6)
        encoded = base64.b64encode(compressed).decode('utf-8')
        print(f"   Gradients: {len(grad_bytes)/1024/1024:.1f}MB → {len(compressed)/1024/1024:.1f}MB")
        return encoded

print("✅ TrainingEngine AILO-1B (memory optimized)")

In [ ]:
#@title 🔄 4. Network Client (con registrazione dashboard)

class NetworkClient:
    def __init__(self, wallet, gpu_info):
        self.wallet = wallet
        self.gpu_info = gpu_info
        self.ws = None
        self.connected = False
        self.session_id = None
    
    async def register(self):
        """Register with server to appear in dashboard."""
        try:
            print("📝 Registering with Ailo server...")
            async with aiohttp.ClientSession() as session:
                async with session.post(f"{API_URL}/cuda/register", json={
                    'wallet': self.wallet,
                    'clientVersion': CLIENT_VERSION,
                    'deviceInfo': {
                        'gpu_name': self.gpu_info.get('gpu_name', 'Tesla T4'),
                        'vram_gb': self.gpu_info.get('vram_gb', 15),
                        'hashrate': 0
                    }
                }) as resp:
                    if resp.status == 200:
                        data = await resp.json()
                        self.session_id = data.get('sessionId')
                        print(f"✅ Registered! Session: {self.session_id[:20]}...")
                        return True
        except Exception as e:
            print(f"⚠️ Registration failed: {e}")
        return False
    
    async def connect(self):
        try:
            print(f"🔌 Connecting to {WS_URL}...")
            self.ws = await websockets.connect(WS_URL, ping_interval=30, ping_timeout=10)
            await self.ws.send(json.dumps({
                'type': 'auth', 'wallet': self.wallet, 'version': CLIENT_VERSION,
                'gpu': self.gpu_info.get('gpu_name', 'Unknown'),
                'vram': self.gpu_info.get('vram_gb', 0)
            }))
            response = await asyncio.wait_for(self.ws.recv(), timeout=10)
            data = json.loads(response)
            if data.get('type') == 'auth_success':
                self.connected = True
                print(f"✅ WebSocket connected!")
                return True
        except Exception as e:
            print(f"⚠️ WebSocket failed: {e}")
        return False
    
    async def send_heartbeat(self, hashrate, status, epoch, loss):
        """Send heartbeat via HTTP for dashboard visibility."""
        try:
            # HTTP ping for dashboard
            async with aiohttp.ClientSession() as session:
                async with session.post(f"{API_URL}/ping", json={
                    'wallet': self.wallet,
                    'status': status,
                    'hashrate': f"{hashrate:.0f} Tok/s"
                }) as resp:
                    pass
            
            # Also update CUDA client stats
            async with aiohttp.ClientSession() as session:
                async with session.post(f"{API_URL}/cuda/register", json={
                    'wallet': self.wallet,
                    'clientVersion': CLIENT_VERSION,
                    'deviceInfo': {
                        'gpu_name': self.gpu_info.get('gpu_name', 'Tesla T4'),
                        'vram_gb': self.gpu_info.get('vram_gb', 15),
                        'hashrate': hashrate
                    }
                }) as resp:
                    pass
            
            # WebSocket heartbeat if connected
            if self.connected and self.ws:
                await self.ws.send(json.dumps({
                    'type': 'heartbeat', 'wallet': self.wallet,
                    'hashrate': hashrate, 'status': status, 'epoch': epoch, 'loss': loss
                }))
        except:
            pass
    
    async def submit_gradients(self, gradients_b64, epoch, loss, hashrate):
        try:
            async with aiohttp.ClientSession() as session:
                async with session.post(f"{API_URL}/cuda/submit", json={
                    'wallet': self.wallet,
                    'gradients': gradients_b64,
                    'epoch': epoch,
                    'loss': loss,
                    'hashrate': hashrate,
                    'gpu': self.gpu_info.get('gpu_name', 'Tesla T4')
                }) as resp:
                    if resp.status == 200:
                        data = await resp.json()
                        reward = data.get('reward', 0)
                        print(f"💰 Reward: {reward:.4f} ALC (Total: {data.get('totalRewards', 0):.4f} ALC)")
                        return reward
                    else:
                        print(f"⚠️ Submit returned {resp.status}")
        except Exception as e:
            print(f"❌ Submit failed: {e}")
        return 0

print("✅ NetworkClient (con registrazione dashboard)")

In [ ]:
#@title 🚀 5. Avvia il Mining!

async def run_mining():
    if not WALLET_ADDRESS or len(WALLET_ADDRESS) < 40:
        print("❌ Inserisci wallet address nella cella 2!")
        return
    
    gc.collect()
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    gpu_info = {
        'gpu_name': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU',
        'vram_gb': torch.cuda.get_device_properties(0).total_memory / 1024**3 if torch.cuda.is_available() else 0
    }
    
    print("="*50)
    print(f"  Ailo GPU Miner v{CLIENT_VERSION} - AILO-1B")
    print("="*50)
    print(f"GPU: {gpu_info['gpu_name']} ({gpu_info['vram_gb']:.1f} GB)")
    print(f"Wallet: {WALLET_ADDRESS[:12]}...{WALLET_ADDRESS[-8:]}\n")
    
    # Initialize network and register FIRST
    network = NetworkClient(WALLET_ADDRESS, gpu_info)
    await network.register()  # Register to appear in dashboard!
    await network.connect()   # Try WebSocket
    
    # Then initialize model
    training = TrainingEngine()
    training.initialize_model()
    
    print("\n⛏️ MINING STARTED!")
    print("📊 Check dashboard: https://ailo.site/dashboard.html\n")
    
    total_steps, total_tokens, total_rewards = 0, 0, 0.0
    start_time = time.time()
    last_heartbeat = last_submit = time.time()
    current_loss, current_hashrate = 0.0, 0.0
    
    try:
        while True:
            step_start = time.time()
            
            # Heartbeat every HEARTBEAT_INTERVAL seconds
            if time.time() - last_heartbeat >= HEARTBEAT_INTERVAL:
                await network.send_heartbeat(current_hashrate, 'training', total_steps, current_loss)
                last_heartbeat = time.time()
            
            batch_x, batch_y = training.generate_training_batch(WALLET_ADDRESS)
            loss, tokens = training.train_step(batch_x, batch_y)
            
            total_steps += 1
            total_tokens += tokens
            current_loss = loss
            elapsed = time.time() - step_start
            current_hashrate = tokens / elapsed if elapsed > 0 else 0
            
            if total_steps % 50 == 0:
                gpu_mem = torch.cuda.memory_allocated() / 1024**3 if torch.cuda.is_available() else 0
                print(f"Step {total_steps} | Loss: {loss:.4f} | {current_hashrate:.0f} tok/s | GPU: {gpu_mem:.1f}GB")
            
            if time.time() - last_submit >= SUBMIT_INTERVAL:
                print("\n📤 Submitting gradients...")
                gradients = training.extract_gradients()
                if gradients:
                    compressed = training.compress_gradients(gradients)
                    reward = await network.submit_gradients(compressed, total_steps, current_loss, current_hashrate)
                    total_rewards += reward
                last_submit = time.time()
                print()
                
    except KeyboardInterrupt:
        print("\n⏹️ Stopped")
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
    finally:
        print(f"\n📊 SUMMARY: {total_steps} steps, {total_tokens:,} tokens, {total_rewards:.4f} ALC")

await run_mining()

---

## ⚠️ Se hai errori di memoria:
1. **Runtime > Disconnect and delete runtime**
2. **Runtime > Run all** per ricominciare pulito

## ℹ️ Info
- **Modello**: AILO-1B (~900M params) - FP16
- **Optimizer**: SGD (usa meno memoria di AdamW)
- **Dashboard**: Vai su https://ailo.site/dashboard.html per vedere il tuo miner!

## 🔗 Links
- [Ailo Network](https://ailo.site)
- [Dashboard](https://ailo.site/dashboard.html)
- [Wallet](https://ailo.site/wallet.html)

---
*Ailo Network - Decentralized AI Training*